In [ ]:
import arcticdb as adb
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import dtale
from sklearn.decomposition import PCA
from tradingo.utils import get_instruments, get_config, with_instrument_details

In [ ]:
a = adb.Arctic("lmdb:///home/rory/dev/airflow/test/arctic.db")
signals = a.get_library("MODEL_SIGNALS")
backtests = a.get_library("BACKTEST")
prices = a.get_library("ASSET_PRICES")

In [ ]:
instruments = get_instruments(get_config())

orig_prices = prices.read("ETFT.ADJ_CLOSE.yfinance")
start_dates = orig_prices.data.apply(lambda i: i.first_valid_index()).rename("start_date")
instruments = instruments.merge(start_dates, left_index=True, right_index=True, how='left')

In [ ]:
PRICES = with_instrument_details((orig_prices.data[
                instruments[(instruments.start_date < pd.Timestamp("2018-03-01"))].index
          ]).dropna(), instruments, columns=["Asset Class"])
RETURNS = np.log(PRICES/PRICES.shift()).dropna()

In [ ]:
PRICES.groupby(["Asset Class"], axis=1).mean().plot()

In [ ]:
(1+RETURNS.loc[:,"Commodities & Digital Assets"].transpose().merge(instruments.Name, left_index=True, right_index=True).reset_index(drop=True).set_index(["Name"]).transpose()).cumprod().plot()

In [ ]:
pca = PCA()
pca.fit(RETURNS)
pd.DataFrame(pca.components_, index=RETURNS.columns)
df = pd.DataFrame(pca.components_, columns=RETURNS.columns)

In [ ]:
signals.list_symbols()

In [ ]:
signals.read("ETFT.trend.ewmac_64_32").data

In [ ]:
dtale.show(backtests.read("ETFT.trend.INSTRUMENT_RETURNS").data)